In [1]:
import pymongo
import pandas as pd
from datetime import datetime
import requests
import time

db_client = pymongo.MongoClient("mongodb://localhost:27017/")
db = db_client["beramminger"]
collection = db["u18_beramminger"]

In [2]:
# Find all documents that contains "ENE" in the "saksnummer" field and have "sakstype" field.
# Request the "sakId" field for each document and use it to make a GET request to the "https://www.domstol.no/api/episerver/v3/beramming/{sakId}" endpoint. 
# If the "sakstype" field in the response is "Fengsling", update the document in the MongoDB collection with {"sakstype": Fengsling} and add a new field "saksinfo" with the entire response data.

document_count = 0
fengsling_count = 0
documents = list(collection.find({"sakstype": {"$exists": False}}))

print(f"Checking {len(documents)} documents for sakstype 'Fengsling'...")

for document in documents:
    document_count += 1
    sak_id = document["sakId"]
    print(f"{document_count}. Sjekker sakId {sak_id}...")

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/145.0.0.0 Safari/537.36',
    }

    response = requests.get(
        'https://www.domstol.no/api/episerver/v3/beramming/{}'.format(sak_id),
        headers=headers,
    )

    sak_data = response.json()

    collection.update_one(
        {"_id": document["_id"]},
        {"$set": {"sakstype": sak_data["sakstype"], "saksinfo": sak_data}}
    )    

    if sak_data["sakstype"] == "Fengsling":
        print("Fengsling!")
        fengsling_count += 1
    
    time.sleep(0.5)

print(f"Updated {fengsling_count} documents with sakstype 'Fengsling'.")

Checking 3723 documents for sakstype 'Fengsling'...
1. Sjekker sakId AAAA2304072130128548105MUECFR_EJBSak...
2. Sjekker sakId AAAA2507221958061974268VIUDYS_EJBSak...
3. Sjekker sakId AAAA2508191527464411014APZBEI_EJBSak...
4. Sjekker sakId AAAA2408210918208701113GPICQV_EJBSak...
5. Sjekker sakId AAAA2509041157390832664FOSBDD_EJBSak...
6. Sjekker sakId AAAA2504251527231801831KBMABG_EJBSak...
7. Sjekker sakId AAAA2502111358003681017DSJFJJ_EJBSak...
8. Sjekker sakId AAAA2504101404114956184SJPQGF_EJBSak...
9. Sjekker sakId AAAA2511171505024622612BBXZIA_EJBSak...
10. Sjekker sakId AAAA2504250824474886163PWBYBQ_EJBSak...
11. Sjekker sakId AAAA2511181602326346132NYWXFU_EJBSak...
12. Sjekker sakId AAAA2412271220545129687WAOJNN_EJBSak...
13. Sjekker sakId AAAA2509141230494522653GVVISC_EJBSak...
14. Sjekker sakId AAAA2510071152306926131DLHWQE_EJBSak...
15. Sjekker sakId AAAA2510291455525712654IPAMGC_EJBSak...
16. Sjekker sakId AAAA2511201016478913087RTUVSQ_EJBSak...
17. Sjekker sakId AAAA2511050

In [3]:
# Find all documents in the collection that has exactly two hours between "startdato" and "sluttdato" and the "saksnm" field contains "ENE"

query = {
    "$expr": {
        "$eq": [
            {"$divide": [{"$subtract": [{"$dateFromString": {"dateString": "$sluttdato"}}, {"$dateFromString": {"dateString": "$startdato"}}]}, 3600000]},
            2
        ]
    },
    "sakstype": "Fengsling"
}

results = list(collection.find(query))

results
# u18_beramminger = []

# for result in results:
#     startdato = datetime.strptime(result["startdato"], "%Y-%m-%dT%H:%M:%S")
#     sluttdato = datetime.strptime(result["sluttdato"], "%Y-%m-%dT%H:%M:%S")
#     differanse = sluttdato - startdato
#     if differanse.total_seconds() == 7200:
#         u18_beramminger.append(result)

# len(u18_beramminger)

[{'_id': ObjectId('6996ce6189773963b42a6f76'),
  'id': '258474',
  'AdvokaterLang': 'Emilie Anine Moe-Hanssen - Gro Arnesen Fremstad, Marius Ihlebæk',
  'DatoIntervaller': [{}],
  'Ids': ['258474'],
  'ParterLang': None,
  'RettensFormann': None,
  'bistandsadvokater': None,
  'domstol': 'Søndre Østfold tingrett',
  'embeteId': 'AAAA2103101754092672012RXHZEG_EJBOrgUnit',
  'parter': None,
  'rettsmoeteIntervaller': [{'start': '06.02.2026', 'end': '06.02.2026'}],
  'sakId': 'AAAA2602061233557291024WDPVLI_EJBSak',
  'sakenGjelder': 'Førstegangsfengsling - U18',
  'saksnummer': '26-022187ENE-TSOS/TSAR',
  'sluttdato': '2026-02-06T15:45:00',
  'startdato': '2026-02-06T13:45:00',
  'saksinfo': {'sakId': 'AAAA2602061233557291024WDPVLI_EJBSak',
   'domstolId': 'AAAA2103101754092672012RXHZEG_EJBOrgUnit',
   'domstol': 'Søndre Østfold tingrett',
   'domstolUrl': '/en/domstoler/tingrett/sondre-ostfold-tingrett/',
   'rettsmotetype': None,
   'saksnummer': '26-022187ENE-TSOS/TSAR',
   'sakstype':